import numpy as np
import cv2
import pyrealsense2 as rs
from utils.Camera.realsense import RealSense

# 카메라 시리얼 번호 (실제 장치에 맞게 수정)
SERIAL_CAM1 = "138322252637"  # 1번 카메라
SERIAL_CAM2 = "f1371347"      # 2번 카메라

class MultiCameraTracker:
    def __init__(self):
        # 두 카메라 초기화
        self.cam1 = RealSense(serial=SERIAL_CAM1)
        self.cam2 = RealSense(serial=SERIAL_CAM2)
        
        # 카메라 설정 (해상도는 동일해야 함)
        self.resolution_color = (640, 480)
        self.resolution_depth = (640, 480)
        
        self.cam1.initialize(resolution_color=self.resolution_color, 
                           resolution_depth=self.resolution_depth)
        self.cam2.initialize(resolution_color=self.resolution_color, 
                           resolution_depth=self.resolution_depth)
        
        # 공 색상 범위 (HSV)
        self.ballLower = (0, 152, 100)
        self.ballUpper = (105, 255, 255)
        
        # 칼만 필터 초기화 (선택 사항)
        self.kf = KalmanFilter()

    def detect_ball(self, img_rgb, img_depth, cam_fx):
        """단일 카메라에서 공 검출"""
        # 이미지 전처리
        img_blur = cv2.GaussianBlur(img_rgb, (11, 11), 0)
        img_hsv = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(img_hsv, self.ballLower, self.ballUpper)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # 컨투어 검출
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        if len(cnts) == 0:
            return None  # 공 검출 실패

        # 가장 큰 컨투어 선택
        c = max(cnts, key=cv2.contourArea)
        ((u, v), r) = cv2.minEnclosingCircle(c)
        
        # 3D 좌표 변환 (픽셀 → 미터)
        z = img_depth[int(v), int(u)]  # 깊이 값 (mm 단위면 1000으로 나눌 것)
        x = (u - self.resolution_color[0]//2) / cam_fx * z
        y = (v - self.resolution_color[1]//2) / cam_fx * z
        
        return np.array([x, y, z])

    def run(self):
        while True:
            # 두 카메라에서 프레임 획득
            rgb1, _, depth1 = self.cam1.get_color_depth('cv')
            rgb2, _, depth2 = self.cam2.get_color_depth('cv')
            
            # 각 카메라에서 공 검출
            pos1 = self.detect_ball(rgb1, depth1, self.cam1._fx)
            pos2 = self.detect_ball(rgb2, depth2, self.cam2._fx)
            
            # 좌표 평균 계산
            if pos1 is not None and pos2 is not None:
                avg_pos = (pos1 + pos2) / 2  # 3D 좌표 평균
                print(f"평균 좌표: X={avg_pos[0]:.2f}m, Y={avg_pos[1]:.2f}m, Z={avg_pos[2]:.2f}m")
            elif pos1 is not None:
                avg_pos = pos1
                print(f"Cam1 Only: X={pos1[0]:.2f}m, Y={pos1[1]:.2f}m, Z={pos1[2]:.2f}m")
            elif pos2 is not None:
                avg_pos = pos2
                print(f"Cam2 Only: X={pos2[0]:.2f}m, Y={pos2[1]:.2f}m, Z={pos2[2]:.2f}m")
            else:
                print("공 검출 실패")
                continue
            
            # 여기에 칼만 필터 적용 가능
            # filtered_pos = self.kf.predict(avg_pos[0], avg_pos[1], avg_pos[2])
            
            # 시각화 (옵션)
            cv2.imshow("Cam1", rgb1)
            cv2.imshow("Cam2", rgb2)
            if cv2.waitKey(1) == 27:
                break

if __name__ == "__main__":
    tracker = MultiCameraTracker()
    tracker.run()
    cv2.destroyAllWindows()